# 신호등 최종 코드

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from PIL import Image

# 🔄 Softmax 함수 정의
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# 🔹 YOLOv8 모델 로드 (COCO pretrained)
yolo_model = YOLO("yolov8n.pt")
TARGET_CLASS_ID = 9  # traffic light

# 🔹 TFLite CNN 모델 로드 (float32, NCHW: [1, 3, 64, 64])
interpreter = tf.lite.Interpreter(model_path="traffic_light_classifier_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 🔹 입력 영상
video_path = "/data/LEE/test1.mp4"
cap = cv2.VideoCapture(video_path)

# 🔹 저장 영상 설정
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("traffic_light_result.mp4", fourcc, fps, (width, height))

# 🔹 프레임 처리 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. YOLO 감지
    results = yolo_model(frame)[0]

    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.cpu().numpy()
        if int(cls) != TARGET_CLASS_ID:
            continue

        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        crop_img = frame[y1:y2, x1:x2]

        if crop_img.size == 0:
            continue

        # ✅ HSV 기반 필터링
        hsv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
        brightness = np.mean(hsv_img[..., 2])        # 밝기 평균
        saturation = np.mean(hsv_img[..., 1])        # 채도 평균
        max_brightness = np.max(hsv_img[..., 2])     # 밝기 최댓값

        if brightness < 40 or saturation < 50 or max_brightness < 80:
            continue  # 공통적으로 너무 어두우면 무시

        # ✅ 초록불과 빨간불 색상 비율 검사
        green_pixels = np.sum((hsv_img[..., 0] > 40) & (hsv_img[..., 0] < 90) & (hsv_img[..., 1] > 50))
        red_pixels = np.sum(((hsv_img[..., 0] < 10) | (hsv_img[..., 0] > 160)) & (hsv_img[..., 1] > 50))

        total_pixels = crop_img.shape[0] * crop_img.shape[1]
        green_ratio = green_pixels / total_pixels
        red_ratio = red_pixels / total_pixels

        if green_ratio < 0.05 and red_ratio < 0.05:
            continue  # 둘 다 너무 적으면 무시

        # 2. CNN 전처리
        img = Image.fromarray(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)).resize((64, 64))
        input_data = np.asarray(img, dtype=np.float32) / 255.0
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = np.expand_dims(input_data, axis=0)

        # 3. 추론
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        probs = softmax(output)
        conf_green, conf_red = float(probs[0]), float(probs[1])

        # ✅ 확률 기반 판단 + green 오탐 방지 강화
        if conf_green > conf_red and conf_green > 0.8:
            if green_ratio < 0.10:  # 🔒 더 엄격한 초록 비중 필터
                continue
            label = "green"
            confidence = conf_green
        elif conf_red > 0.8:
            if red_ratio < 0.05:  # 🔒 빨강도 최소한의 비중은 필요
                continue
            label = "red"
            confidence = conf_red
        else:
            continue  # 확신 부족

        # 4. 시각화
        color = (0, 255, 0) if label == "green" else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        text = f"{label.upper()} ({confidence:.2f})"
        cv2.putText(frame, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 프레임 저장
    out.write(frame)

# 마무리
cap.release()
out.release()
print("✅ 결과 저장 완료: traffic_light_result.mp4")


# 횡단보도

In [8]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZxSEfCzVHgJMRddrxT0u")
project = rf.workspace("tom-lai-8bp7n").project("crosswalk-xlzhu")
version = project.version(6)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Crosswalk-6 in yolov8:: 100%|██████████| 2999/2999 [00:00<00:00, 7544.24it/s]


In [10]:
import os
import cv2
import shutil

def is_dark(image_path, threshold=50):
    img = cv2.imread(image_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    v_mean = hsv[..., 2].mean()
    return v_mean < threshold

# 📁 경로 설정
image_dir = "/data/LEE/Crosswalk-6/train/images"
label_dir = "/data/LEE/Crosswalk-6/train/labels"

# 📁 제거할 목록 저장
remove_list = []

for filename in os.listdir(image_dir):
    if not filename.endswith(".jpg"):
        continue
    img_path = os.path.join(image_dir, filename)
    if is_dark(img_path):
        remove_list.append(filename)

# 🔥 이미지 + 레이블 같이 제거
for fname in remove_list:
    img_path = os.path.join(image_dir, fname)
    label_path = os.path.join(label_dir, fname.replace(".jpg", ".txt"))
    os.remove(img_path)
    if os.path.exists(label_path):
        os.remove(label_path)

print(f"✅ 총 {len(remove_list)}개의 어두운 이미지가 제거되었습니다.")


✅ 총 94개의 어두운 이미지가 제거되었습니다.


In [9]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZxSEfCzVHgJMRddrxT0u")
project = rf.workspace("mia-lkjrc").project("safet_crosswalk")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to safeT_Crosswalk-1 in yolov8:: 100%|██████████| 2440/2440 [00:00<00:00, 8524.48it/s]


## 횡단보도 학습(Crosswalk-6)

In [11]:
from ultralytics import YOLO

# ✅ 모델 불러오기 (경량 버전)
model = YOLO("yolov8n.pt")  # yolov8s.pt 등으로 변경 가능

# ✅ 학습 수행
model.train(
    data="/data/LEE/Crosswalk-6/data.yaml",  # ← 실제 경로에 맞게 수정
    epochs=100,
    imgsz=640,
    batch=16,               # GPU 사양에 맞게 조절
    project="/data/LEE/Crosswalk-6",         # 결과 폴더 위치
    name="crosswalk_train", # 결과 폴더 이름
    exist_ok=True           # 폴더 중복 방지
)

# ✅ 학습 완료 후 파일 경로
best_model_path = "/data/LEE/Crosswalk-6/crosswalk.pt"
print(f"✅ 모델 학습 완료! crosswalk.pt 저장 위치:\n{best_model_path}")


New https://pypi.org/project/ultralytics/8.3.145 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.94 🚀 Python-3.8.10 torch-1.12.1+cu113 CUDA:0 (NVIDIA A100-SXM4-80GB MIG 1g.10gb, 9728MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/data/LEE/Crosswalk-6/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/LEE/Crosswalk-6, name=crosswalk_train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, vi

100%|██████████| 5.35M/5.35M [00:00<00:00, 11.7MB/s]


AMP: checks passed ✅


train: Scanning /data/LEE/Crosswalk-6/train/labels... 1191 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1191/1191 [00:00<00:00, 1634.80it/s]

train: New cache created: /data/LEE/Crosswalk-6/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1290, len(boxes) = 1444. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    100/100      2.65G     0.5254     0.3702      1.105          8        640: 100%|██████████| 75/75 [00:11<00:00,  6.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.58it/s]

                   all        210        259      0.956      0.931      0.964      0.795



100 epochs completed in 0.353 hours.
Optimizer stripped from /data/LEE/Crosswalk-6/crosswalk_train/weights/last.pt, 6.3MB
Optimizer stripped from /data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt, 6.3MB

Validating /data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.8.10 torch-1.12.1+cu113 CUDA:0 (NVIDIA A100-SXM4-80GB MIG 1g.10gb, 9728MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  3.60it/s]


                   all        210        259      0.966      0.919      0.965      0.797
Speed: 0.7ms preprocess, 3.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /data/LEE/Crosswalk-6/crosswalk_train
✅ 모델 학습 완료! crosswalk.pt 저장 위치:
/data/LEE/Crosswalk-6/crosswalk.pt


## 횡단보도 출력(/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt)

In [13]:
import cv2
from ultralytics import YOLO

# ✅ 모델 로드
model = YOLO("/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt")  # ← best.pt 경로 확인

# ✅ 영상 경로 설정
video_path = "/data/LEE/test1.mp4"         # ⬅️ 추론할 영상 경로
output_path = "result_crosswalk.mp4"  # ⬅️ 출력 영상 경로

# ✅ 비디오 읽기
cap = cv2.VideoCapture(video_path)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# ✅ 비디오 저장 객체 초기화
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# ✅ 프레임별 추론
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 추론 (이미지 단일 프레임 입력)
    results = model(frame)[0]

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        if conf < 0.5:
            continue

        # 바운딩 박스 좌표 정수화
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # 바운딩 박스 및 텍스트 표시
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        label = f"Crosswalk {conf:.2f}"
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # 결과 프레임 저장
    out.write(frame)

# ✅ 마무리
cap.release()
out.release()
print("🎬 추론 완료! 결과 영상 저장됨:", output_path)



0: 640x384 4 crosswalks, 8.7ms
Speed: 2.2ms preprocess, 8.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 8.3ms
Speed: 1.2ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 8.1ms
Speed: 1.2ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 11.4ms
Speed: 1.5ms preprocess, 11.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 7.8ms
Speed: 1.2ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 crosswalks, 7.8ms
Speed: 1.2ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 crosswalks, 7.8ms
Speed: 1.1ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 crosswalks, 7.9ms
Speed: 1.2ms preprocess, 7.9ms inference, 1.0ms postprocess per image 

# 신호등 + 횡단보도(최종)

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from PIL import Image

# 🔄 Softmax 함수
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# ✅ 모델 불러오기
traffic_yolo = YOLO("yolov8n.pt")  # 신호등 감지 (COCO pretrained)
crosswalk_yolo = YOLO("/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt")  # 횡단보도 전용 모델
TARGET_CLASS_ID = 9  # traffic light 클래스 (COCO 기준)

# ✅ TFLite 신호등 색 분류기 로드
interpreter = tf.lite.Interpreter(model_path="traffic_light_classifier_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ 입력 영상
video_path = "/data/LEE/test2.mp4"
cap = cv2.VideoCapture(video_path)

# ✅ 출력 영상 설정
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("final_result2.mp4", fourcc, fps, (width, height))

# ✅ 프레임 처리 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. YOLO로 신호등 감지
    results = traffic_yolo(frame)[0]
    red_light_detected = False

    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.cpu().numpy()
        if int(cls) != TARGET_CLASS_ID:
            continue

        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size == 0:
            continue

        # HSV 필터링
        hsv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
        brightness = np.mean(hsv_img[..., 2])
        saturation = np.mean(hsv_img[..., 1])
        max_brightness = np.max(hsv_img[..., 2])
        if brightness < 40 or saturation < 50 or max_brightness < 80:
            continue

        # 색상 비율
        green_pixels = np.sum((hsv_img[..., 0] > 40) & (hsv_img[..., 0] < 90) & (hsv_img[..., 1] > 50))
        red_pixels = np.sum(((hsv_img[..., 0] < 10) | (hsv_img[..., 0] > 160)) & (hsv_img[..., 1] > 50))
        total_pixels = crop_img.shape[0] * crop_img.shape[1]
        green_ratio = green_pixels / total_pixels
        red_ratio = red_pixels / total_pixels
        if green_ratio < 0.05 and red_ratio < 0.05:
            continue

        # CNN 분류기 전처리 및 추론
        img = Image.fromarray(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)).resize((64, 64))
        input_data = np.asarray(img, dtype=np.float32) / 255.0
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = np.expand_dims(input_data, axis=0)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        probs = softmax(output)
        conf_green, conf_red = float(probs[0]), float(probs[1])

        # 색상 판별
        if conf_green > conf_red and conf_green > 0.8:
            if green_ratio < 0.10:
                continue
            label = "green"
            confidence = conf_green
        elif conf_red > 0.8:
            if red_ratio < 0.05:
                continue
            label = "red"
            confidence = conf_red
            red_light_detected = True
        else:
            continue

        # 신호등 시각화
        color = (0, 255, 0) if label == "green" else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        text = f"{label.upper()} ({confidence:.2f})"
        cv2.putText(frame, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 2. 횡단보도는 항상 감지
    import cv2
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from PIL import Image

# 🔄 Softmax 함수
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# ✅ 모델 불러오기
traffic_yolo = YOLO("yolov8n.pt")  # 신호등 감지 (COCO pretrained)
crosswalk_yolo = YOLO("/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt")  # 횡단보도 전용 모델
TARGET_CLASS_ID = 9  # traffic light 클래스 (COCO 기준)

# ✅ TFLite 신호등 색 분류기 로드
interpreter = tf.lite.Interpreter(model_path="traffic_light_classifier_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ 입력 영상
video_path = "/data/LEE/test2.mp4"
cap = cv2.VideoCapture(video_path)

# ✅ 출력 영상 설정
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("final_result2.mp4", fourcc, fps, (width, height))

# ✅ 프레임 처리 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. YOLO로 신호등 감지
    results = traffic_yolo(frame)[0]
    red_light_detected = False

    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.cpu().numpy()
        if int(cls) != TARGET_CLASS_ID:
            continue

        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size == 0:
            continue

        # HSV 필터링
        hsv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
        brightness = np.mean(hsv_img[..., 2])
        saturation = np.mean(hsv_img[..., 1])
        max_brightness = np.max(hsv_img[..., 2])
        if brightness < 40 or saturation < 50 or max_brightness < 80:
            continue

        # 색상 비율
        green_pixels = np.sum((hsv_img[..., 0] > 40) & (hsv_img[..., 0] < 90) & (hsv_img[..., 1] > 50))
        red_pixels = np.sum(((hsv_img[..., 0] < 10) | (hsv_img[..., 0] > 160)) & (hsv_img[..., 1] > 50))
        total_pixels = crop_img.shape[0] * crop_img.shape[1]
        green_ratio = green_pixels / total_pixels
        red_ratio = red_pixels / total_pixels
        if green_ratio < 0.05 and red_ratio < 0.05:
            continue

        # CNN 분류기 전처리 및 추론
        img = Image.fromarray(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)).resize((64, 64))
        input_data = np.asarray(img, dtype=np.float32) / 255.0
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = np.expand_dims(input_data, axis=0)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        probs = softmax(output)
        conf_green, conf_red = float(probs[0]), float(probs[1])

        # 색상 판별
        if conf_green > conf_red and conf_green > 0.8:
            if green_ratio < 0.10:
                continue
            label = "green"
            confidence = conf_green
        elif conf_red > 0.8:
            if red_ratio < 0.05:
                continue
            label = "red"
            confidence = conf_red
            red_light_detected = True
        else:
            continue

        # 신호등 시각화
        color = (0, 255, 0) if label == "green" else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        text = f"{label.upper()} ({confidence:.2f})"
        cv2.putText(frame, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 2. 횡단보도는 항상 감지
    cross_results = crosswalk_yolo(frame)[0]
    for cbox in cross_results.boxes.data.tolist():
        cx1, cy1, cx2, cy2, cconf, ccls = cbox
        if cconf < 0.5:
            continue
        cx1, cy1, cx2, cy2 = map(int, [cx1, cy1, cx2, cy2])
        cv2.rectangle(frame, (cx1, cy1), (cx2, cy2), (255, 255, 0), 2)
        cv2.putText(frame, f"Crosswalk ({cconf:.2f})", (cx1, cy1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)


    # 프레임 저장
    out.write(frame)
    

# 마무리
cap.release()
out.release()
print("✅ 최종 결과 저장 완료: final_result2.mp4")



0: 640x384 5 cars, 7.2ms
Speed: 1.2ms preprocess, 7.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 7.3ms
Speed: 1.0ms preprocess, 7.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 6 cars, 7.2ms
Speed: 1.0ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 7.1ms
Speed: 1.0ms preprocess, 7.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 6 cars, 7.2ms
Speed: 1.0ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 5 cars, 7.2ms
Speed: 1.0ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 cars, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 6 cars, 7.2ms
Speed: 1.0ms preprocess, 7.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 cars, 7.1m

# + 가로등

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import tensorflow as tf

# 🔄 Softmax 함수
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# ✅ 모델 로딩
traffic_yolo = YOLO("yolov8n.pt")  # 신호등 감지 (COCO)
crosswalk_yolo = YOLO("/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt")  # 횡단보도
lamp_yolo = YOLO("/data/LEE/obstacle_detection_epoch15.pt")  # 가로등 감지 모델 (사용자 준비 필요)
TARGET_CLASS_ID = 9  # traffic light 클래스 (COCO 기준)

# ✅ TFLite 신호등 색 분류기
interpreter = tf.lite.Interpreter(model_path="traffic_light_classifier_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ 영상 설정
video_path = "/data/LEE/test1.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("final_result_with_streetlamp.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# ✅ 프레임 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    red_light_detected = False
    results = traffic_yolo(frame)[0]

    # 1. 신호등 감지 + 색 분류
    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.cpu().numpy()
        if int(cls) != TARGET_CLASS_ID:
            continue
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size == 0:
            continue

        hsv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
        brightness = np.mean(hsv_img[..., 2])
        saturation = np.mean(hsv_img[..., 1])
        max_brightness = np.max(hsv_img[..., 2])
        if brightness < 40 or saturation < 50 or max_brightness < 80:
            continue

        green_pixels = np.sum((hsv_img[..., 0] > 40) & (hsv_img[..., 0] < 90) & (hsv_img[..., 1] > 50))
        red_pixels = np.sum(((hsv_img[..., 0] < 10) | (hsv_img[..., 0] > 160)) & (hsv_img[..., 1] > 50))
        total_pixels = crop_img.shape[0] * crop_img.shape[1]
        green_ratio = green_pixels / total_pixels
        red_ratio = red_pixels / total_pixels
        if green_ratio < 0.05 and red_ratio < 0.05:
            continue

        img = Image.fromarray(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)).resize((64, 64))
        input_data = np.asarray(img, dtype=np.float32) / 255.0
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = np.expand_dims(input_data, axis=0)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        probs = softmax(output)
        conf_green, conf_red = float(probs[0]), float(probs[1])

        if conf_green > conf_red and conf_green > 0.8:
            if green_ratio < 0.10: continue
            label, color, confidence = "green", (0, 255, 0), conf_green
        elif conf_red > 0.8:
            if red_ratio < 0.05: continue
            label, color, confidence = "red", (0, 0, 255), conf_red
            red_light_detected = True
        else:
            continue

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label.upper()} ({confidence:.2f})", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 2. 횡단보도 감지
    cross_results = crosswalk_yolo(frame)[0]
    for cbox in cross_results.boxes.data.tolist():
        cx1, cy1, cx2, cy2, cconf, ccls = cbox
        if cconf < 0.5: continue
        cx1, cy1, cx2, cy2 = map(int, [cx1, cy1, cx2, cy2])
        cv2.rectangle(frame, (cx1, cy1), (cx2, cy2), (255, 255, 0), 2)
        cv2.putText(frame, f"Crosswalk ({cconf:.2f})", (cx1, cy1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    # 3. 가로등 감지
    lamp_results = lamp_yolo(frame)[0]
    for lbox in lamp_results.boxes.data.tolist():
        lx1, ly1, lx2, ly2, lconf, lcls = lbox
        if lconf < 0.5: continue
        lx1, ly1, lx2, ly2 = map(int, [lx1, ly1, lx2, ly2])
        cv2.rectangle(frame, (lx1, ly1), (lx2, ly2), (255, 0, 255), 2)
        cv2.putText(frame, f"StreetLamp ({lconf:.2f})", (lx1, ly1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)

    # 프레임 저장
    out.write(frame)

# ✅ 마무리
cap.release()
out.release()
print("✅ 최종 결과 저장 완료: final_result_with_streetlamp.mp4")



0: 640x384 1 person, 3 cars, 1 bus, 6 traffic lights, 7.8ms
Speed: 2.2ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 7.5ms
Speed: 1.3ms preprocess, 7.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 street_pole, 8.0ms
Speed: 1.2ms preprocess, 8.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 2 cars, 2 buss, 5 traffic lights, 8.0ms
Speed: 1.3ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4 crosswalks, 8.1ms
Speed: 1.2ms preprocess, 8.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 street_pole, 7.8ms
Speed: 1.1ms preprocess, 7.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 3 cars, 2 buss, 5 traffic lights, 8.1ms
Speed: 1.2ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 4

# 모든 객체 감지

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import tensorflow as tf

# 🔄 Softmax 함수
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# ✅ 모델 로딩
traffic_yolo = YOLO("yolov8m.pt")  # 신호등 감지 (COCO)
crosswalk_yolo = YOLO("/data/LEE/Crosswalk-6/crosswalk_train/weights/best.pt")  # 횡단보도
lamp_yolo = YOLO("/data/LEE/obstacle_detection_epoch15.pt")  # 가로등 감지 모델 (사용자 준비 필요)
TARGET_CLASS_ID = 9  # traffic light 클래스 (COCO 기준)

# ✅ TFLite 신호등 색 분류기
interpreter = tf.lite.Interpreter(model_path="traffic_light_classifier_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# ✅ 영상 설정
video_path = "/data/LEE/2.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("2_final.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# ✅ 프레임 루프
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    red_light_detected = False
    results = traffic_yolo(frame)[0]

    # 1. 신호등 감지 + 색 분류
    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.cpu().numpy()
        if int(cls) != TARGET_CLASS_ID:
            continue
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        crop_img = frame[y1:y2, x1:x2]
        if crop_img.size == 0:
            continue

        hsv_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
        brightness = np.mean(hsv_img[..., 2])
        saturation = np.mean(hsv_img[..., 1])
        max_brightness = np.max(hsv_img[..., 2])
        if brightness < 40 or saturation < 50 or max_brightness < 80:
            continue

        green_pixels = np.sum((hsv_img[..., 0] > 40) & (hsv_img[..., 0] < 90) & (hsv_img[..., 1] > 50))
        red_pixels = np.sum(((hsv_img[..., 0] < 10) | (hsv_img[..., 0] > 160)) & (hsv_img[..., 1] > 50))
        total_pixels = crop_img.shape[0] * crop_img.shape[1]
        green_ratio = green_pixels / total_pixels
        red_ratio = red_pixels / total_pixels
        if green_ratio < 0.05 and red_ratio < 0.05:
            continue

        img = Image.fromarray(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)).resize((64, 64))
        input_data = np.asarray(img, dtype=np.float32) / 255.0
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = np.expand_dims(input_data, axis=0)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])[0]
        probs = softmax(output)
        conf_green, conf_red = float(probs[0]), float(probs[1])

        if conf_green > conf_red and conf_green > 0.8:
            if green_ratio < 0.10: continue
            label, color, confidence = "green", (0, 255, 0), conf_green
        elif conf_red > 0.8:
            if red_ratio < 0.05: continue
            label, color, confidence = "red", (0, 0, 255), conf_red
            red_light_detected = True
        else:
            continue

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label.upper()} ({confidence:.2f})", (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # 2. 횡단보도 감지
    cross_results = crosswalk_yolo(frame)[0]
    for cbox in cross_results.boxes.data.tolist():
        cx1, cy1, cx2, cy2, cconf, ccls = cbox
        if cconf < 0.85: continue  # ✅ 0.9 이상일 때만 처리
        cx1, cy1, cx2, cy2 = map(int, [cx1, cy1, cx2, cy2])
        cv2.rectangle(frame, (cx1, cy1), (cx2, cy2), (255, 255, 0), 2)
        cv2.putText(frame, f"Crosswalk ({cconf:.2f})", (cx1, cy1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)



    # 3. 사용자 커스텀 YOLO 모델의 모든 객체 감지
    lamp_results = lamp_yolo(frame)[0]
    for lbox in lamp_results.boxes.data.tolist():
        lx1, ly1, lx2, ly2, lconf, lcls = lbox
        if lconf < 0.5:
            continue
        lx1, ly1, lx2, ly2 = map(int, [lx1, ly1, lx2, ly2])
        class_id = int(lcls)
        class_name = lamp_yolo.model.names[class_id] if class_id in lamp_yolo.model.names else f"Class {class_id}"
        cv2.rectangle(frame, (lx1, ly1), (lx2, ly2), (255, 0, 255), 2)
        cv2.putText(frame, f"{class_name} ({lconf:.2f})", (lx1, ly1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        
    # 4. 자전거 감지 (COCO 모델 사용)
    # 4. 자전거 감지 (사용자 커스텀 모델 사용, 클래스 번호 1번만 감지)
    bike_yolo = YOLO("/data/LEE/bike-and-braille-3/bicycle_train/weights/best.pt")

    bike_results = bike_yolo(frame)[0]  # ✅ bike_yolo는 사용자 학습된 YOLO 객체여야 함

    for bbox in bike_results.boxes.data.tolist():
        bx1, by1, bx2, by2, bconf, bcls = bbox
        if int(bcls) != 1:  # ✅ 클래스 번호가 1번만 감지
            continue
        if bconf < 0.5:
            continue
        bx1, by1, bx2, by2 = map(int, [bx1, by1, bx2, by2])
        cv2.rectangle(frame, (bx1, by1), (bx2, by2), (0, 255, 255), 2)
        cv2.putText(frame, f"Custom Bicycle ({bconf:.2f})", (bx1, by1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)



    # 프레임 저장
    out.write(frame)

# ✅ 마무리
cap.release()
out.release()
print("✅ 최종 결과 저장 완료: 2_final.mp4")



0: 640x384 (no detections), 20.4ms
Speed: 2.1ms preprocess, 20.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.2ms
Speed: 1.2ms preprocess, 7.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.1ms
Speed: 1.2ms preprocess, 7.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.3ms
Speed: 1.2ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 19.8ms
Speed: 1.1ms preprocess, 19.8ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 6.9ms
Speed: 1.0ms preprocess, 6.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.0ms
Speed: 1.2ms preprocess, 7.0ms inference, 0.

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f14df15e070>>
Traceback (most recent call last):
  File "/data/yolov8_env/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 



0: 640x384 (no detections), 7.1ms
Speed: 1.2ms preprocess, 7.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bus, 15.6ms
Speed: 1.1ms preprocess, 15.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 crosswalk, 7.2ms
Speed: 1.2ms preprocess, 7.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.1ms
Speed: 1.2ms preprocess, 7.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.3ms
Speed: 1.2ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 bus, 16.0ms
Speed: 1.2ms preprocess, 16.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 crosswalk, 7.1ms
Speed: 1.1ms preprocess, 7.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 7.1ms
Speed: 1.1ms preprocess, 7.1ms inference, 0.4ms po

KeyboardInterrupt: 

# 버스 번호 인식

In [ ]:
import cv2
import easyocr
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import re  # 숫자 검출을 위한 정규 표현식 사용

# **1️⃣ YOLO 모델 로드**
model_path = "/data/BUS/trained_model.pt"  # YOLO 모델 파일 경로
model = YOLO(model_path)

# **2️⃣ 영상 로드 및 저장 준비**
video_path = "/data/LEE/버스2.mp4"  # 입력 비디오 경로
output_video_path = "/data/LEE/버스2_final.mp4"  # 출력 비디오 경로
cap = cv2.VideoCapture(video_path)  # 비디오 캡처 객체

# 비디오 속성 가져오기
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 비디오 저장을 위한 VideoWriter 객체 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# **3️⃣ EasyOCR 리더 생성**
reader = easyocr.Reader(["ko", "en"], gpu=False)  # GPU 사용 안 함

# **4️⃣ YOLO를 사용하여 객체 탐지 및 OCR 적용**
target_classes = {"front_num", "back_num", "side_num"}  # ✅ bus 제외

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    cropped_images = {}
    expand_ratio = 0.1  # 🔹 바운딩 박스를 확장할 비율 (10%)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 검출된 박스 좌표
            label = r.names[int(box.cls[0])]  # 객체 이름

            if label in target_classes:  # ✅ 번호판 3개만 처리
                # 🔹 바운딩 박스 확장
                width, height = x2 - x1, y2 - y1
                x1 = max(0, x1 - int(width * expand_ratio))
                y1 = max(0, y1 - int(height * expand_ratio))
                x2 = min(frame.shape[1], x2 + int(width * expand_ratio))
                y2 = min(frame.shape[0], y2 + int(height * expand_ratio))

                cropped_img = frame[y1:y2, x1:x2]
                cropped_images[label] = cropped_img  # 라벨별 이미지 저장

    bus_number = "N/A (모든 번호 인식 실패)"
    bus_number_source = None  # 어디서 인식했는지 저장
    processed_images = {}

    for label, cropped_img in cropped_images.items():
        # 🔹 1차 OCR을 위한 이미지 전처리 (이진화)
        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        # 🔹 밝은 숫자 강조 (THRESH_BINARY)
        lower_white = 200
        upper_white = 255
        _, binary = cv2.threshold(gray, lower_white, upper_white, cv2.THRESH_BINARY)

        # 🔹 OCR 실행 (이진화된 이미지)
        ocr_result = reader.readtext(binary, detail=0)
        extracted_text = " ".join(ocr_result) if ocr_result else "N/A"

        # 🔹 OCR 결과가 숫자를 포함하는지 확인
        if re.search(r"\d", extracted_text):  # 숫자가 하나라도 포함된 경우
            bus_number = extracted_text  # 버스 번호 확정
            bus_number_source = f"{label} (이진화된 이미지)"
            processed_images = {label: binary}  # 해당 이미지만 출력
            break  # 첫 번째로 감지된 숫자로 확정 후 반복 종료

    # **5️⃣ 영상에 OCR 결과 표시**
    for label, cropped_img in cropped_images.items():
        if label in processed_images:
            # OCR을 적용한 이미지에 텍스트 추가
            cv2.putText(frame, bus_number, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # **6️⃣ 영상 저장**
    out.write(frame)

# **7️⃣ 영상 처리 종료 및 저장**
cap.release()
out.release()

print(f"🎯 최종 버스 번호: {bus_number} (출처: {bus_number_source})")


/data/yolov8_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


TypeError: 'weights_only' is an invalid keyword argument for load()